In [ ]:
from typing import List

from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate


In [ ]:
%env OPENAI_API_KEY=

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

vectorstore = FAISS.load_local(folder_path="../embeddings", embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr")

template = """You are an assistant for question-answering tasks.
You will be asked questions about XRootD, or eXtended Request Daemon.
Use the following pieces of retrieved context to answer the question.
Use your existing knowledge to answer the question if the retrieved context
does not contain useful information.
Do not mention the context to the user. Do now let user know you are given a retrieved context.
If you don't know the answer or nothing is provided in the context, just say: Sorry, I don't know.

{context}

Question: {question}

Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:

def format_docs(docs: List[Document]):
    return "\n\n".join(doc.page_content for doc in docs)

def extract_sources(docs: List[Document]):
    sources = []
    for doc in docs:
        sources.append(doc.metadata.get("source"))
    return sources

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
answer = rag_chain_with_source.invoke("What is oss in xrootd?")
answer["context"] = extract_sources(answer["context"])
answer